In [1]:
from google.colab import drive
drive.mount('/content/data') # 드라이브 연결

Mounted at /content/data


In [2]:
%matplotlib inline
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [13]:
class NeuralNetwork(nn.Module): # nn.module  상속받아서 자식 class 만듬(객체화)
    def __init__(self):
        super(NeuralNetwork, self).__init__() #super를 통해 부모 생성 
        self.flatten = nn.Flatten() # 데이터를 1차원으로 만들어줌 
        self.linear_relu_stack = nn.Sequential( # 순서형으로 실행
            nn.Linear(28*28, 512), # input = 28,28 (정해진 이미지 사이즈-> 데이터shape) / 512개 output 입력 
            nn.ReLU(), #active function
            nn.Linear(512, 512),  # input = 512 / output = 512
            nn.ReLU(),
            nn.Linear(512, 10),  # input = 512 / output=10  -> 분류를 10가지로 하는 모델(target shape)
            nn.ReLU()
        ) # fulli conneceted layer 3개

    def forward(self, x):  # x(28,28)를 통해 이미지 한장씩 들어옴(모델 학습=이미지를 보고 예측 반복)
        out_flatten = self.flatten(x) # 데이터를 1차원으로 만들어줌 
        logits = self.linear_relu_stack(out_flatten) # logits: (10,)
        return logits

In [21]:
model = NeuralNetwork().to(device) # NeuralNetwork을 cpu로
print(model) 

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [12]:
X = torch.rand(1,28,28,device=device) # 모델에 넣을 input 만듬
X.shape

torch.Size([1, 28, 28])

In [15]:
logits = model(X) 
logits.shape 

torch.Size([1, 10])

In [16]:
logits

tensor([[0.0000, 0.1689, 0.0000, 0.0000, 0.0399, 0.0638, 0.0000, 0.0000, 0.0479,
         0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)

In [17]:
pred_probab = nn.Softmax(dim=1)(logits) # softmax 적용
pred_probab.shape

torch.Size([1, 10])

In [18]:
pred_probab

tensor([[0.0967, 0.1145, 0.0967, 0.0967, 0.1007, 0.1031, 0.0967, 0.0967, 0.1015,
         0.0967]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [20]:
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


## Weight and Bias

In [22]:
print(f"1 Linear weights: {model.linear_relu_stack[0].weight.shape} \n") # (output , input) = (512,28*28)
print('-'*50)
print(f"1 Linear weights: {model.linear_relu_stack[0].bias.shape} \n") # (512) = output = 노드개수 

1 Linear weights: torch.Size([512, 784]) 

--------------------------------------------------
1 Linear weights: torch.Size([512]) 



In [23]:
print(f"2 Linear weights: {model.linear_relu_stack[2].weight.shape} \n") # (512, 512)
print('-'*50)
print(f"2 Linear weights: {model.linear_relu_stack[2].bias.shape} \n") # (512, 512)

2 Linear weights: torch.Size([512, 512]) 

--------------------------------------------------
2 Linear weights: torch.Size([512]) 



In [24]:
print(f"3 Linear weights: {model.linear_relu_stack[4].weight.shape} \n") # (10, 512)
print('-'*50)
print(f"3 Linear weights: {model.linear_relu_stack[4].bias.shape} \n") # (10)

3 Linear weights: torch.Size([10, 512]) 

--------------------------------------------------
3 Linear weights: torch.Size([10]) 



## Model layers(뜯어보기)

In [28]:
input_image =torch.rand(5,28,28) #배치 사이즈 5(무시가능) , 28*28 사이즈 데이터 만듬
print(input_image.size())

torch.Size([5, 28, 28])


In [29]:
flatten = nn.Flatten()

flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([5, 784])


In [30]:
layer1 = nn.Linear(in_features=28*28,out_features=20) #linear 생성
hidden1 = layer1(flat_image) # (5,28*28)
print(hidden1.size())

torch.Size([5, 20])


In [31]:
hidden1 = nn.ReLU()(hidden1) # 음수를 다 0으로 처리 

In [32]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10) #input = 20 , output = 10
) # 순서대로 실행해서 생성 

input_image = torch.rand(3,28,28)

logits = seq_modules(input_image) #적용

In [34]:
softmax = nn.Softmax(dim=1) # 확률로 표현을 해준다
pred_probab = softmax(logits)
pred_probab.shape 

torch.Size([3, 10])

## Model parameters

In [35]:
print("Model structure: ", model, "\n\n")

for i, (name, param) in enumerate(model.named_parameters()):
    print(f"{i}번째, Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


0번째, Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0018, -0.0321, -0.0297,  ..., -0.0042, -0.0084,  0.0012],
        [ 0.0106, -0.0281, -0.0148,  ..., -0.0160, -0.0271, -0.0219]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

1번째, Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0086, -0.0232], device='cuda:0', grad_fn=<SliceBackward0>) 

2번째, Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0197, -0.0389, -0.0362,  ...,  0.0174,  0.0160, -0.0335],
        [-0.0271, -0.0005,  0.0057,  ..., -0.0343,  0.0080,  0.0428]],
  